<a href="https://colab.research.google.com/github/MahaleVivek/RAG-based-chat-application/blob/main/Viamagus_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install openai pinecone-client sentence-transformers tiktoken

In [ ]:
# Add all impports here
import os
import time
import openai
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import torch
import tiktoken

In [ ]:
# Variable definitions
OPENAI_API_KEY = "<YOUR_OPENAI_API_KEY>"
PINECONE_API_KEY = "<YOUR_PINECONE_API_KEY>"
PINECONE_CLOUD = '<SELECTED_PINECONE_CLOUD>'
PINECONE_REGION = '<SELECTED_PINECONE_REGION>'

In [ ]:
# Initialize OpenAI client
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_CLOUD)

In [ ]:
MAX_TOKENS = 225
PINECONE_INDEX_NAME = 'chat-history'

In [ ]:
# Define the conversation history

history = [
 "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
 "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
 "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
 "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
 "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
 "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
 "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
 "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
 "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
 "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
 "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
 "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
 "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
 "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
 "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
 "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
 "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
 "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
 "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
 "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]

In [ ]:
# Initialize the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)

In [ ]:
existing_indexes = [
index_info["name"] for index_info in pc.list_indexes()
]

existing_indexes

[]

In [ ]:
# Define the function to add embeddings to Pinecone
def add_embeddings_to_pinecone(history, index_name='chat-history'):
    """
    This function should:
    1. Encode each message in the history using a transformer model.
    2. Include the message number as part of the embedding.
    3. Create a Pinecone index if it does not exist.
    4. Upsert the encoded messages into the Pinecone index.
    """
    # check if index already exists (it shouldn't if this is first time)
    if PINECONE_INDEX_NAME not in pc.list_indexes().names():
        print(f"\nTrue, {PINECONE_INDEX_NAME} is not present so creating one...")
        # if does not exist, create index
        pc.create_index(
            PINECONE_INDEX_NAME,
            dimension=384,  # dimensionality of minilm
            metric='cosine',
            spec=ServerlessSpec(
                cloud=PINECONE_CLOUD,
                region=PINECONE_REGION
                )
            )
        # wait for index to be initialized
        while not pc.describe_index(PINECONE_INDEX_NAME).status['ready']:
            time.sleep(1)

    # Connect to the index
    index = pc.Index(PINECONE_INDEX_NAME)

    # Prepare the data to upsert
    data = []
    for i, entry in enumerate(history, start=1):
        _, message = entry.split('| Bot:', 1)
        embedding = model.encode(message.strip()).tolist()
        data.append((str(i), embedding, {'text': message.strip()}))


    # Upsert the data
    index.upsert(data)

In [ ]:
add_embeddings_to_pinecone(history)


True, chat-history is not present so creating one...


In [ ]:
# Define the function to retrieve relevant history
def retrieve_relevant_history(query, index_name='chat-history', top_k=5):
    """
    This function should:
    1. Encode the query using a transformer model.
    2. Query the Pinecone index with the encoded query to retrieve the
       most relevant messages.
    3. Return the relevant messages.
    """
    # Encode the query
    query_embedding = model.encode(query).tolist()

    # Connect to the index
    index = pc.Index(PINECONE_INDEX_NAME)

    # Query the index
    result = index.query(vector=query_embedding, include_metadata=True, top_k=top_k)

    # Extract relevant messages from the query result
    relevant_messages = [match['metadata']['text'] for match in result['matches']]

    return relevant_messages

In [ ]:
def prepare_prompt(test_prompt, history, index_name='chat-history', max_tokens=2048):
    """
    This function should:
    1. Retrieve relevant history messages using the RAG mechanism.
    2. Combine the retrieved messages with the test prompt.
    3. Ensure the combined prompt does not exceed the maximum token
       limit.
    4. Return the combined prompt and the context referred.
    """
    # Retrieve relevant history messages
    relevant_history = retrieve_relevant_history(test_prompt, index_name=index_name)

    # Combine the retrieved messages with the test prompt
    combined_prompt = "\n\n".join(relevant_history) + "\n\n" + test_prompt

    # Tokenize the combined prompt
    encoding = tiktoken.get_encoding('cl100k_base')  # Replace with the appropriate model name if different
    encoded_prompt = encoding.encode(combined_prompt)
    encoded_prompt = str(encoded_prompt)

    # Convert the string representation of the list to an actual list
    encoded_prompt_list = eval(encoded_prompt)

    # Ensure the combined prompt does not exceed the maximum token limit
    if len(encoded_prompt_list) > max_tokens:
        # Truncate the history messages if needed
        while len(encoded_prompt_list) > max_tokens:
            relevant_history.pop(0)  # Remove the oldest message
            combined_prompt = "\n\n".join(relevant_history) + "\n\n" + test_prompt
            encoded_prompt = encoding.encode(combined_prompt)

    # Return the combined prompt and the context referred
    return combined_prompt, relevant_history

In [ ]:
# Define the function to test the final prompt
def test_final_prompt():
    """
    This function should:
    1. Define the final test prompt.
    2. Prepare the prompt using the prepare_prompt function.
    3. Call OpenAI to generate a response.
    4. Print the final test prompt, the chat history context referred to, and the final response.
    5. Validate manually if the final response references the correct context.
    """
    final_test_prompt = "Do you think it will help me stay fit?"

    # Prepare the prompt using the prepare_prompt function
    prepared_prompt, context_referred = prepare_prompt(final_test_prompt, history)

    # Get the chat completion response from OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                {"role": "user", "content": f"Context: {context_referred}\n\n---\n\nQuestion: {final_test_prompt}\nAnswer:"}
            ],
        max_tokens=MAX_TOKENS,
        temperature=0,
        n=1,
        stop=None
    )

    print(f"\nFinal Test Prompt: {final_test_prompt}")
    print(f"\nContext Referred: {context_referred}")
    print(f"\nFinal Test Prompt Response: {response.choices[0].message.content}")

In [ ]:
test_final_prompt()


Final Test Prompt: Do you think it will help me stay fit?

Context Referred: ['You can try a mix of cardio and strength training exercises.', 'Running, cycling, and swimming are all excellent cardio exercises.', 'Roll your clothes to save space and use packing cubes.', 'Sure! How about trying a new salad recipe?', 'Just enjoy your journey and make the most out of your experiences.']

Final Test Prompt Response: Yes, trying a mix of cardio and strength training exercises like running, cycling, and swimming can help you stay fit.
